# 🚀 ITOS - Intraday Trading Operating System (Google Colab Edition)

## Complete Workflow on Google Colab

This notebook demonstrates the complete ITOS workflow on Google Colab:
1. **Mount Google Drive** and setup environment
2. **Convert Kaggle data** to Parquet format
3. **Run trading strategies** across all symbols
4. **Analyze performance** with comprehensive metrics
5. **Generate visualizations** and recommendations

In [ ]:
# Install dependencies (run once)
!pip install pandas numpy pyarrow matplotlib seaborn tqdm kagglehub
!pip install --upgrade google-api-python-client google-colab

In [ ]:
# Mount Google Drive and setup environment
from google.colab import drive
import os
import sys
import gc
import logging
from datetime import datetime

# Mount Google Drive
print("🔗 Mounting Google Drive...")
drive.mount('/content/drive')

# Setup paths
DRIVE_BASE = "/content/drive/MyDrive/itos"
DATA_DIR = f"{DRIVE_BASE}/minute_parquet"
RESULTS_DIR = f"{DRIVE_BASE}/STR_results"
ANALYTICS_DIR = f"{DRIVE_BASE}/analytics"
LOGS_DIR = f"{DRIVE_BASE}/logs"

# Create directories
for directory in [DATA_DIR, RESULTS_DIR, ANALYTICS_DIR, LOGS_DIR]:
    os.makedirs(directory, exist_ok=True)

print(f"✅ Google Drive mounted and directories created")
print(f"📁 Base Directory: {DRIVE_BASE}")

# Add project to Python path
sys.path.insert(0, DRIVE_BASE)

print("🎯 Environment setup completed")

## Step 1: Download and Convert Kaggle Data

In [ ]:
# Download Kaggle dataset
import kagglehub

DATASET = "debashis74017/algo-trading-data-nifty-100-data-with-indicators"
print("📥 Downloading Kaggle dataset...")

path = kagglehub.dataset_download(DATASET)
print(f"📁 Dataset downloaded at: {path}")

# List downloaded files
import os
print(f"📋 Files in dataset: {len(os.listdir(path))}")
print(f"📄 Sample files: {os.listdir(path)[:5]}")

In [ ]:
# Convert CSV data to Parquet format
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

SOURCE_DIR = path
TARGET_DIR = DATA_DIR

# Get list of CSV files
csv_files = sorted([f for f in os.listdir(SOURCE_DIR) if f.endswith("_minute.csv")])
print(f"📊 Found {len(csv_files)} CSV files to convert")

# Check existing files
existing_files = set([f.replace('.parquet', '') for f in os.listdir(TARGET_DIR) if f.endswith('.parquet')])
files_to_convert = [f.replace('_minute.csv', '') for f in csv_files if f.replace('_minute.csv', '') not in existing_files]

print(f"📈 Files to convert: {len(files_to_convert)}")
print(f"⏭ Already converted: {len(existing_files)}")

def detect_timestamp_column(df):
    """Detect timestamp column in DataFrame."""
    timestamp_columns = ['timestamp', 'datetime', 'date', 'Date', 'time', 'Time']
    for col in timestamp_columns:
        if col in df.columns:
            return col
    return None

def process_csv_to_parquet(csv_file, symbol):
    """Process a single CSV file to Parquet format."""
    try:
        csv_path = os.path.join(SOURCE_DIR, csv_file)
        parquet_path = os.path.join(TARGET_DIR, f"{symbol}.parquet")
        
        # Read CSV in chunks
        chunks = []
        chunk_size = 10000
        
        for chunk in pd.read_csv(csv_path, chunksize=chunk_size, low_memory=False):
            # Detect timestamp column
            ts_col = detect_timestamp_column(chunk)
            if ts_col is None:
                print(f"⚠️ No timestamp column found for {symbol}")
                return False
            
            # Rename and convert timestamp
            chunk = chunk.rename(columns={ts_col: 'timestamp'})
            chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], errors='coerce')
            
            # Remove invalid rows
            chunk = chunk.dropna(subset=['timestamp'])
            chunk = chunk.sort_values('timestamp')
            
            # Filter trading hours
            chunk['time'] = chunk['timestamp'].dt.time
            trading_hours = (
                (chunk['time'] >= pd.Timestamp('09:15').time()) & 
                (chunk['time'] <= pd.Timestamp('15:30').time())
            )
            chunk = chunk[trading_hours].drop('time', axis=1)
            
            # Standardize columns
            chunk.columns = chunk.columns.str.lower().str.replace(' ', '_')
            
            # Map required columns
            column_mapping = {
                'open': 'open', 'high': 'high', 'low': 'low', 'close': 'close', 'volume': 'volume'
            }
            
            for old_name, new_name in column_mapping.items():
                if old_name in chunk.columns:
                    chunk = chunk.rename(columns={old_name: new_name})
            
            # Check required columns
            required_cols = ['open', 'high', 'low', 'close', 'volume']
            if not all(col in chunk.columns for col in required_cols):
                print(f"⚠️ Missing required columns for {symbol}")
                return False
            
            # Add basic features
            chunk['returns'] = chunk['close'].pct_change()
            chunk['typical_price'] = (chunk['high'] + chunk['low'] + chunk['close']) / 3
            chunk['range'] = chunk['high'] - chunk['low']
            chunk['body'] = abs(chunk['close'] - chunk['open'])
            
            chunks.append(chunk)
        
        if not chunks:
            print(f"⚠️ No valid data for {symbol}")
            return False
        
        # Combine chunks and save
        df = pd.concat(chunks, ignore_index=True)
        df.to_parquet(parquet_path, compression='snappy', index=False)
        
        del df, chunks
        gc.collect()
        
        return True
        
    except Exception as e:
        print(f"❌ Error processing {symbol}: {e}")
        return False

# Convert files (with progress bar)
successful = []
failed = []

for i, symbol in enumerate(tqdm(files_to_convert, desc="Converting files")):
    csv_file = f"{symbol}_minute.csv"
    
    if process_csv_to_parquet(csv_file, symbol):
        successful.append(symbol)
    else:
        failed.append(symbol)
    
    # Memory cleanup
    if i % 10 == 0:
        gc.collect()

print(f"\n🎉 Conversion completed:")
print(f"   ✅ Successful: {len(successful)}")
print(f"   ❌ Failed: {len(failed)}")
print(f"   📊 Total processed: {len(successful) + len(existing_files)}")

## Step 2: Load ITOS System and Run Strategies

In [ ]:
# Import ITOS modules
from itos_colab.core.data_engine import ColabDataEngine
from itos_colab.core.strategy_engine import ColabStrategyEngine
from itos_colab.core.analytics_engine import ColabAnalyticsEngine
from itos_colab.strategies.str001_orb import STR001_ORB
from itos_colab.strategies.str002_orb_filtered import STR002_ORB_FILTERED
from itos_colab.strategies.str003_vwap_pullback import STR003_VWAP_PULLBACK

print("✅ ITOS modules imported successfully")

# Initialize data engine and get summary
data_engine = ColabDataEngine()
summary = data_engine.get_data_summary()

print(f"\n📊 Data Summary:")
print(f"   Symbols: {summary.get('symbols', 0)}")
print(f"   Records: {summary.get('total_records', 0):,}")
print(f"   Date Range: {summary.get('date_range', ('N/A', 'N/A'))}")

In [ ]:
# Preview data for one symbol
symbols = data_engine.get_processed_symbols()

if symbols:
    test_symbol = symbols[0]
    market_data = data_engine.load_symbol_data(test_symbol)
    
    if market_data:
        print(f"📈 Sample data for {test_symbol}:")
        print(f"   Shape: {market_data.data.shape}")
        print(f"   Columns: {list(market_data.data.columns)}")
        print(f"   Date range: {market_data.data['timestamp'].min()} to {market_data.data['timestamp'].max()}")
        
        # Display sample data
        display(market_data.data.head())
        
        # Plot price chart
        import matplotlib.pyplot as plt
        plt.figure(figsize=(12, 6))
        plt.plot(market_data.data['timestamp'], market_data.data['close'])
        plt.title(f'{test_symbol} - Price Chart')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print(f"❌ No data found for {test_symbol}")
else:
    print("❌ No symbols found")

## Step 3: Run Trading Strategies

In [ ]:
# Initialize strategy engine
strategy_engine = ColabStrategyEngine()

# Define strategies to run
strategies = [
    STR001_ORB(),
    STR002_ORB_FILTERED(),
    STR003_VWAP_PULLBACK()
]

print(f"🚀 Running {len(strategies)} strategies...")

# Get symbols to process
symbols = data_engine.get_processed_symbols()
print(f"📊 Available symbols: {len(symbols)}")

# For testing, limit to first 20 symbols (remove this for full run)
# symbols = symbols[:20]
# print(f"🧪 Testing with first {len(symbols)} symbols")

In [ ]:
# Run each strategy
results = []

for strategy in strategies:
    strategy_name = strategy.get_config().name
    print(f"\n📈 Running {strategy_name}...")
    
    # Run strategy
    success = strategy_engine.run_strategy(strategy, symbols)
    
    if success:
        # Load and display results
        trades_df = strategy_engine.get_strategy_results(strategy_name)
        
        if trades_df is not None and not trades_df.empty:
            print(f"   ✅ {strategy_name} completed with {len(trades_df)} trades")
            
            # Basic statistics
            total_pnl = trades_df['pnl'].sum()
            winning_trades = len(trades_df[trades_df['pnl'] > 0])
            win_rate = winning_trades / len(trades_df) * 100
            
            print(f"   📊 Total PnL: ₹{total_pnl:,.2f}")
            print(f"   🎯 Win Rate: {win_rate:.1f}%")
            print(f"   📈 Symbols: {trades_df['symbol'].nunique()}")
            
            # Display sample trades
            print(f"   📋 Sample trades:")
            display(trades_df.head())
            
            results.append({
                'strategy': strategy_name,
                'trades': len(trades_df),
                'pnl': total_pnl,
                'win_rate': win_rate,
                'symbols': trades_df['symbol'].nunique()
            })
        else:
            print(f"   ⚠️ {strategy_name} completed but no trades generated")
    else:
        print(f"   ❌ {strategy_name} failed")
    
    # Memory cleanup
    gc.collect()

print(f"\n🎉 Strategy execution completed!")

## Step 4: Analytics and Performance Evaluation

In [ ]:
# Initialize analytics engine
analytics_engine = ColabAnalyticsEngine()

print(f"📊 Running comprehensive analytics...")

# Analyze each strategy
detailed_results = []

for strategy in strategies:
    strategy_name = strategy.get_config().name
    print(f"\n🔍 Analyzing {strategy_name}...")
    
    # Load trades
    trades_df = strategy_engine.get_strategy_results(strategy_name)
    
    if trades_df is not None and not trades_df.empty:
        # Run comprehensive analytics
        result = analytics_engine.analyze_strategy(strategy_name, trades_df)
        detailed_results.append(result)
        
        # Print detailed results
        print(f"\n📋 {strategy_name} Detailed Analytics:")
        print(f"   Total Trades: {result.total_trades}")
        print(f"   Winning Trades: {result.winning_trades}")
        print(f"   Losing Trades: {result.losing_trades}")
        print(f"   Total PnL: ₹{result.total_pnl:,.2f}")
        print(f"   Max Drawdown: {result.max_drawdown*100:.2f}%")
        print(f"   Profit Factor: {result.profit_factor:.2f}")
        print(f"   Sharpe Ratio: {result.sharpe_ratio:.2f}")
        print(f"   Avg Duration: {result.avg_trade_duration:.1f} minutes")
        
        # Get recommendation
        recommendation = analytics_engine.evaluate_strategy(result)
        print(f"   Recommendation: {recommendation}")
        
        if recommendation == "APPROVE":
            print("   ✅ Strategy approved for live trading")
        elif recommendation == "MODIFY":
            print("   ⚠️ Strategy needs modification")
        else:
            print("   ❌ Strategy killed - not viable")
        
        # Display additional insights
        if result.metadata:
            print(f"\n💡 Additional Insights:")
            
            if 'win_rate_analysis' in result.metadata:
                win_data = result.metadata['win_rate_analysis']
                print(f"   Overall Win Rate: {win_data['overall_win_rate']*100:.1f}%")
                print(f"   Symbol Avg Win Rate: {win_data['symbol_avg_win_rate']*100:.1f}%")
            
            if 'risk_metrics' in result.metadata:
                risk_data = result.metadata['risk_metrics']
                print(f"   VaR (95%): ₹{risk_data.get('var_95', 0):,.2f}")
                print(f"   Max Consecutive Losses: {risk_data.get('max_consecutive_losses', 0)}")
                print(f"   Avg Win/Loss Ratio: {risk_data.get('avg_win_to_loss_ratio', 0):.2f}")
    else:
        print(f"   ⚠️ No trades found for {strategy_name}")
    
    # Memory cleanup
    gc.collect()

## Step 5: Strategy Comparison and Visualization

In [ ]:
# Compare strategies
if detailed_results:
    strategy_names = [r.strategy_name for r in detailed_results]
    comparison_df = analytics_engine.compare_strategies(strategy_names)
    
    print(f"\n📊 Strategy Performance Comparison:")
    display(comparison_df)
    
    # Create comparison charts
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Strategy Comparison Dashboard', fontsize=16)
    
    # Total PnL comparison
    axes[0, 0].bar(comparison_df['Strategy'], comparison_df['Total PnL'])
    axes[0, 0].set_title('Total PnL')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].grid(True)
    
    # Win Rate comparison
    axes[0, 1].bar(comparison_df['Strategy'], comparison_df['Win Rate'] * 100)
    axes[0, 1].set_title('Win Rate (%)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True)
    
    # Profit Factor comparison
    axes[1, 0].bar(comparison_df['Strategy'], comparison_df['Profit Factor'])
    axes[1, 0].set_title('Profit Factor')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True)
    
    # Sharpe Ratio comparison
    axes[1, 1].bar(comparison_df['Strategy'], comparison_df['Sharpe Ratio'])
    axes[1, 1].set_title('Sharpe Ratio')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Display best strategy
    best_strategy = comparison_df.iloc[0]
    print(f"\n🏆 Best Strategy: {best_strategy['Strategy']}")
    print(f"   Total PnL: ₹{best_strategy['Total PnL']:,.2f}")
    print(f"   Win Rate: {best_strategy['Win Rate']*100:.1f}%")
    print(f"   Profit Factor: {best_strategy['Profit Factor']:.2f}")
    print(f"   Sharpe Ratio: {best_strategy['Sharpe Ratio']:.2f}")
else:
    print("❌ No strategy results to compare")

## Step 6: Detailed Analysis of Best Strategy

In [ ]:
# Analyze best performing strategy in detail
if detailed_results:
    # Find best strategy by total PnL
    best_result = max(detailed_results, key=lambda x: x.total_pnl)
    best_strategy_name = best_result.strategy_name
    
    print(f"\n🏆 Detailed Analysis of Best Strategy: {best_strategy_name}")
    print(f"   Total PnL: ₹{best_result.total_pnl:,.2f}")
    
    # Load trades for detailed analysis
    trades_df = strategy_engine.get_strategy_results(best_strategy_name)
    
    if trades_df is not None and not trades_df.empty:
        # Convert datetime columns
        trades_df['entry_time'] = pd.to_datetime(trades_df['entry_time'])
        trades_df['exit_time'] = pd.to_datetime(trades_df['exit_time'])
        
        # Monthly performance
        trades_df['entry_month'] = trades_df['entry_time'].dt.to_period('M')
        monthly_stats = trades_df.groupby('entry_month').agg({
            'pnl': ['count', 'sum', 'mean'],
            'symbol': 'nunique'
        }).round(2)
        
        print(f"\n📅 Monthly Performance:")
        monthly_stats.columns = ['Trades', 'Total PnL', 'Avg PnL', 'Symbols']
        display(monthly_stats)
        
        # Symbol performance
        symbol_stats = trades_df.groupby('symbol').agg({
            'pnl': ['count', 'sum', 'mean']
        }).round(2)
        
        symbol_stats.columns = ['Trades', 'Total PnL', 'Avg PnL']
        symbol_stats = symbol_stats.sort_values('Total PnL', ascending=False)
        
        print(f"\n📈 Top Performing Symbols (Top 10):")
        display(symbol_stats.head(10))
        
        # Exit reason analysis
        exit_reason_stats = trades_df.groupby('exit_reason').agg({
            'pnl': ['count', 'sum', 'mean'],
        }).round(2)
        
        exit_reason_stats.columns = ['Count', 'Total PnL', 'Avg PnL']
        
        print(f"\n🚪 Exit Reason Analysis:")
        display(exit_reason_stats)
        
        # Plot equity curve
        trades_df_sorted = trades_df.sort_values('entry_time')
        trades_df_sorted['cumulative_pnl'] = trades_df_sorted['pnl'].cumsum()
        
        plt.figure(figsize=(15, 8))
        plt.subplot(2, 1, 1)
        plt.plot(trades_df_sorted['entry_time'], trades_df_sorted['cumulative_pnl'])
        plt.title(f'{best_strategy_name} - Equity Curve')
        plt.ylabel('Cumulative PnL')
        plt.grid(True)
        
        # Plot drawdown
        plt.subplot(2, 1, 2)
        trades_df_sorted['running_max'] = trades_df_sorted['cumulative_pnl'].expanding().max()
        trades_df_sorted['drawdown'] = trades_df_sorted['cumulative_pnl'] - trades_df_sorted['running_max']
        
        plt.fill_between(trades_df_sorted['entry_time'], trades_df_sorted['drawdown'], 0, 
                         color='red', alpha=0.3)
        plt.plot(trades_df_sorted['entry_time'], trades_df_sorted['drawdown'], 'r-', label='Drawdown')
        plt.title('Drawdown')
        plt.xlabel('Date')
        plt.ylabel('Drawdown')
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
else:
    print("❌ No strategy results available for detailed analysis")

## Final Summary and Next Steps

In [ ]:
print("\n" + "="*80)
print("🎉 ITOS Google Colab Execution Completed!")
print("="*80)

# System summary
print(f"\n📊 System Summary:")
print(f"   📁 Google Drive Base: {DRIVE_BASE}")
print(f"   📈 Processed Symbols: {len(symbols)}")
print(f"   🎯 Strategies Tested: {len(strategies)}")
print(f"   📊 Analytics Generated: {len(detailed_results)}")

if detailed_results:
    # Strategy summary
    print(f"\n📈 Strategy Results:")
    for result in detailed_results:
        recommendation = analytics_engine.evaluate_strategy(result)
        status = "✅" if recommendation == "APPROVE" else "⚠️" if recommendation == "MODIFY" else "❌"
        print(f"   {status} {result.strategy_name}: ₹{result.total_pnl:,.2f} ({result.total_trades} trades)")
    
    # Best strategy
    best_result = max(detailed_results, key=lambda x: x.total_pnl)
    print(f"\n🏆 Best Strategy: {best_result.strategy_name}")
    print(f"   Total PnL: ₹{best_result.total_pnl:,.2f}")
    print(f"   Win Rate: {best_result.winning_trades/best_result.total_trades*100:.1f}%")
    print(f"   Profit Factor: {best_result.profit_factor:.2f}")
    print(f"   Sharpe Ratio: {best_result.sharpe_ratio:.2f}")

# File locations
print(f"\n📁 Generated Files on Google Drive:")
print(f"   📊 Data: {DATA_DIR}")
print(f"   📈 Results: {RESULTS_DIR}")
print(f"   📋 Analytics: {ANALYTICS_DIR}")
print(f"   📄 Logs: {LOGS_DIR}")

print(f"\n🎯 Next Steps:")
print(f"   1. 📊 Review detailed analytics in Google Drive")
print(f"   2. 📈 Examine strategy dashboards (PNG files)")
print(f"   3. 🔧 Modify/kill strategies based on performance")
print(f"   4. 🚀 Develop new strategies using the framework")
print(f"   5. 📈 Test with additional market data")
print(f"   6. 🔄 Implement Phase 3 - Strategy Library")

print(f"\n💡 Key Features Used:")
print(f"   ✅ Google Drive persistence")
print(f"   ✅ Resume-safe processing")
print(f"   ✅ Memory-efficient batch processing")
print(f"   ✅ Comprehensive analytics")
print(f"   ✅ Interactive visualization")
print(f"   ✅ Strategy comparison")

print("\n" + "="*80)
print("🎯 Ready for production use!")
print("="*80)